In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix
)


In [2]:
df_init = pd.read_excel("db\Registro simplificado de participaciones en instancias externas (ETL).xlsx")

print(df_init.columns.tolist())
print(df_init.head().to_markdown())

<>:1: SyntaxWarning: "\R" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\R"? A raw string is also an option.
<>:1: SyntaxWarning: "\R" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\R"? A raw string is also an option.
C:\Users\pablo\AppData\Local\Temp\ipykernel_18644\976382202.py:1: SyntaxWarning: "\R" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\R"? A raw string is also an option.
  df_init = pd.read_excel("db\Registro simplificado de participaciones en instancias externas (ETL).xlsx")


['Id', 'Dependencia', 'Subdependencia', 'Participación', 'Modalidad de Participación', 'País', 'Ciudad', 'Fecha inicio Actividad', 'Fecha termino Actividad', 'Total Días Actividad', 'Ámbitos Estratégicos_Cambio Climático y Mitigación de Impactos Ambientales', 'Ámbitos Estratégicos_Colaboración Intersectorial y Alianzas para el Desarrollo Sostenible', 'Ámbitos Estratégicos_Conservación de la Biodiversidad y Restauración de Ecosistemas', 'Ámbitos Estratégicos_Cultura, Arte y Patrimonio', 'Ámbitos Estratégicos_Deporte, Actividad Física y Bienestar', 'Ámbitos Estratégicos_Desarrollo Laboral y Empleo Digno', 'Ámbitos Estratégicos_Desarrollo Social y Erradicación de la Pobreza', 'Ámbitos Estratégicos_Desarrollo de Tecnologías Limpias y Eficiencia Energética', 'Ámbitos Estratégicos_Educación de Calidad y Equitativa', 'Ámbitos Estratégicos_Educación para el Desarrollo Sostenible', 'Ámbitos Estratégicos_Gestión Sostenible del Agua y los Recursos Hídricos', 'Ámbitos Estratégicos_Inclusión, Diver

In [3]:
df_clasif = df_init.copy()

cols_obligatorias = [
    'Total Días Actividad',
    'Participación',
    'Modalidad de Participación',
    'País',
    'Total ODS',
    'Total Ámbitos Estratégicos'
]

df_clasif.dropna(subset=cols_obligatorias, inplace=True)


In [4]:
df_clasif['Actividad_Larga'] = (df_clasif['Total Días Actividad'] >= 30).astype(int)

umbral = df_clasif['Total Días Actividad'].quantile(0.75)

df_clasif['Actividad_Larga'] = (
    df_clasif['Total Días Actividad'] >= umbral
).astype(int)

y = df_clasif['Actividad_Larga']


In [5]:
df_clasif['Actividad_Larga'].value_counts(normalize=True)


Actividad_Larga
0    0.699115
1    0.300885
Name: proportion, dtype: float64

In [6]:
num_features = [
    'Total ODS',
    'Total Ámbitos Estratégicos'
]

cat_features = [
    'Participación',
    'Modalidad de Participación',
    'País'
]

X = df_clasif[
    num_features +
    cat_features +
    ['Sede_Santiago', 'Sede_Temuco']
]


In [7]:
X = pd.get_dummies(X, drop_first=True)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y  # MUY importante para clasificación
)


In [9]:
scaler = MinMaxScaler()

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[num_features] = scaler.fit_transform(X_train[num_features])
X_test_scaled[num_features] = scaler.transform(X_test[num_features])


In [10]:
model = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',  # útil si hay desbalance
    random_state=42
)

model.fit(X_train_scaled, y_train)

print("Modelo entrenado correctamente.")


Modelo entrenado correctamente.


In [11]:
y_pred = model.predict(X_test_scaled)

In [12]:
accuracy = accuracy_score(y_test, y_pred)

print("\n--- Métricas de Evaluación ---")
print(f"Accuracy: {accuracy:.3f}\n")

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))



--- Métricas de Evaluación ---
Accuracy: 0.870

Matriz de Confusión:
[[14  2]
 [ 1  6]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.93      0.88      0.90        16
           1       0.75      0.86      0.80         7

    accuracy                           0.87        23
   macro avg       0.84      0.87      0.85        23
weighted avg       0.88      0.87      0.87        23



In [13]:
coeficientes = pd.Series(
    model.coef_[0],
    index=X_train_scaled.columns
).sort_values(ascending=False)

print("\n--- Variables que AUMENTAN la probabilidad de actividad larga ---")
print(coeficientes.head(10))

print("\n--- Variables que REDUCEN la probabilidad de actividad larga ---")
print(coeficientes.tail(10))



--- Variables que AUMENTAN la probabilidad de actividad larga ---
País_Francia                                                    1.054341
País_Estados Unidos                                             1.005911
Modalidad de Participación_mesa redonda donecia en Fisiatria    0.976475
País_República Dominicana                                       0.912988
Modalidad de Participación_Organización                         0.905781
Modalidad de Participación_Moderador                            0.677087
País_Brasil                                                     0.564375
Modalidad de Participación_Póster                               0.514653
País_Grecia                                                     0.438719
País_Irlanda                                                    0.438719
dtype: float64

--- Variables que REDUCEN la probabilidad de actividad larga ---
Modalidad de Participación_STAND          -0.350404
Modalidad de Participación_CONFERENCIAS   -0.350404
País_México       

In [14]:
y_prob = model.predict_proba(X_test_scaled)[:, 1]

df_resultados = X_test_scaled.copy()
df_resultados['Probabilidad_Actividad_Larga'] = y_prob
df_resultados['Actividad_Larga_Real'] = y_test.values

df_resultados.head()


,Total ODS,Total Ámbitos Estratégicos,Sede_Santiago,Sede_Temuco,Modalidad de Participación_CHARLA Y PRESENTACIÓN CON DIAPOSITIVAS,Modalidad de Participación_CONFERENCIAS,Modalidad de Participación_Comité Científico Organizador,Modalidad de Participación_Comité científico,Modalidad de Participación_DEI officer,Modalidad de Participación_Entrevista Diario Las Ultimas Noticias,...,País_Irlanda,País_Italia,País_México,País_Perú,País_República Dominicana,País_Santiago,País_Sudáfrica,País_chile,Probabilidad_Actividad_Larga,Actividad_Larga_Real
18,0.000000,0.2,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0.235602,0
64,1.333333,0.6,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0.148086,0
16,0.000000,0.0,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0.806512,0
36,0.000000,0.0,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0.300447,0
149,0.000000,0.4,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0.514841,0
